In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

# Arguments

In [2]:
data_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [3]:
pretrained = ''
resume = './log/style1/64x64/b0/checkpoint.pth.tar'

In [4]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 200
start_epoch = 0
train_batch = 512
test_batch = 500
lr = 0.01
schedule = [70, 150]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/64x64/b0' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (64, 64)

best_acc = 0

In [5]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [6]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes, override_params={'dropout_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4, nesterov=True)
# optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

==> Resuming from checkpoint..


# Train

In [13]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [16]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [3 | 200] LR: 0.024000
1/135 Data:5.104 | Batch:8.394 | Total:0:00:08 | ETA:0:18:45 | Loss:0.694968044757843 | top1:50.9765625
11/135 Data:0.011 | Batch:0.324 | Total:0:00:14 | ETA:0:03:01 | Loss:0.7066751285032793 | top1:49.041194915771484
21/135 Data:2.244 | Batch:2.732 | Total:0:00:29 | ETA:0:02:42 | Loss:0.7029163723900205 | top1:49.600074768066406
31/135 Data:0.007 | Batch:0.510 | Total:0:00:41 | ETA:0:02:06 | Loss:0.7009502187851937 | top1:49.747982025146484
41/135 Data:4.161 | Batch:4.703 | Total:0:00:55 | ETA:0:02:11 | Loss:0.6995354600068999 | top1:50.16672897338867
51/135 Data:0.004 | Batch:0.504 | Total:0:01:05 | ETA:0:01:30 | Loss:0.6995296805512672 | top1:50.18765640258789
61/135 Data:2.984 | Batch:3.476 | Total:0:01:17 | ETA:0:01:31 | Loss:0.6992719427483981 | top1:50.19851303100586
71/135 Data:0.001 | Batch:0.524 | Total:0:01:28 | ETA:0:01:11 | Loss:0.699509580370406 | top1:50.1182861328125
81/135 Data:0.306 | Batch:0.823 | Total:0:01:40 | ETA:0:01:04 | Loss:0.69

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


16/16 Data:0.002 | Batch:0.852 | Total:0:00:21 | ETA:0:00:00 | Loss:0.7037428884934156 | top1:50.0

Epoch: [4 | 200] LR: 0.017000
1/135 Data:6.302 | Batch:6.876 | Total:0:00:06 | ETA:0:15:22 | Loss:0.6947821378707886 | top1:53.125
11/135 Data:0.003 | Batch:0.530 | Total:0:00:17 | ETA:0:03:28 | Loss:0.6942317919297651 | top1:51.278411865234375
21/135 Data:2.608 | Batch:3.118 | Total:0:00:29 | ETA:0:02:21 | Loss:0.695244122119177 | top1:50.716148376464844
31/135 Data:0.003 | Batch:0.478 | Total:0:00:38 | ETA:0:01:35 | Loss:0.6950529390765775 | top1:51.20967483520508
41/135 Data:0.171 | Batch:0.701 | Total:0:00:49 | ETA:0:01:37 | Loss:0.695219932532892 | top1:51.0003776550293
51/135 Data:0.003 | Batch:0.533 | Total:0:01:02 | ETA:0:01:49 | Loss:0.6953504190725439 | top1:51.0493278503418
61/135 Data:2.278 | Batch:2.779 | Total:0:01:15 | ETA:0:01:37 | Loss:0.6952984420979609 | top1:50.99897384643555
71/135 Data:0.016 | Batch:0.536 | Total:0:01:25 | ETA:0:01:06 | Loss:0.6953677140491109 | top

111/135 Data:0.167 | Batch:0.677 | Total:0:02:05 | ETA:0:00:29 | Loss:0.7016936572822364 | top1:54.203617095947266
121/135 Data:0.003 | Batch:0.528 | Total:0:02:17 | ETA:0:00:17 | Loss:0.6996085486136192 | top1:54.461517333984375
131/135 Data:1.588 | Batch:2.119 | Total:0:02:28 | ETA:0:00:05 | Loss:0.7018060188257057 | top1:54.33563995361328
16/16 Data:0.002 | Batch:0.120 | Total:0:00:17 | ETA:0:00:00 | Loss:0.6625794317477789 | top1:60.769229888916016

Epoch: [9 | 200] LR: 0.052000
1/135 Data:6.586 | Batch:7.097 | Total:0:00:07 | ETA:0:15:52 | Loss:0.6735458970069885 | top1:56.8359375
11/135 Data:2.241 | Batch:2.749 | Total:0:00:18 | ETA:0:02:19 | Loss:0.6772046793590892 | top1:57.90127944946289
21/135 Data:0.002 | Batch:0.518 | Total:0:00:30 | ETA:0:02:25 | Loss:0.7000014015606472 | top1:55.598960876464844
31/135 Data:3.722 | Batch:4.223 | Total:0:00:45 | ETA:0:02:36 | Loss:0.6993938415281234 | top1:55.70186233520508
41/135 Data:0.005 | Batch:0.505 | Total:0:00:57 | ETA:0:01:53 | Los

81/135 Data:0.001 | Batch:0.559 | Total:0:01:36 | ETA:0:00:47 | Loss:0.6420755268614969 | top1:63.220970153808594
91/135 Data:2.306 | Batch:2.792 | Total:0:01:49 | ETA:0:00:55 | Loss:0.6396567290955848 | top1:63.46154022216797
101/135 Data:0.003 | Batch:0.492 | Total:0:01:59 | ETA:0:00:36 | Loss:0.637969977784865 | top1:63.598388671875
111/135 Data:1.484 | Batch:1.959 | Total:0:02:11 | ETA:0:00:28 | Loss:0.636838578963065 | top1:63.580379486083984
121/135 Data:0.002 | Batch:0.517 | Total:0:02:21 | ETA:0:00:16 | Loss:0.6356307696705022 | top1:63.684783935546875
131/135 Data:2.805 | Batch:3.330 | Total:0:02:34 | ETA:0:00:06 | Loss:0.6341635416482241 | top1:63.804569244384766
16/16 Data:0.002 | Batch:0.114 | Total:0:00:19 | ETA:0:00:00 | Loss:0.5548922083316705 | top1:72.9102554321289

Epoch: [14 | 200] LR: 0.080000
1/135 Data:4.188 | Batch:4.703 | Total:0:00:04 | ETA:0:10:31 | Loss:0.6248844861984253 | top1:65.234375
11/135 Data:0.008 | Batch:0.521 | Total:0:00:17 | ETA:0:03:30 | Loss:0.

...

81/135 Data:0.553 | Batch:1.146 | Total:0:01:04 | ETA:0:00:41 | Loss:0.25486972633703253 | top1:89.03598022460938
91/135 Data:0.014 | Batch:0.777 | Total:0:01:13 | ETA:0:00:39 | Loss:0.255039054613847 | top1:89.07109069824219
101/135 Data:0.006 | Batch:0.783 | Total:0:01:21 | ETA:0:00:28 | Loss:0.2550450916042422 | top1:89.06636810302734
111/135 Data:0.006 | Batch:0.772 | Total:0:01:29 | ETA:0:00:19 | Loss:0.25458041719488195 | top1:89.05018615722656
121/135 Data:0.007 | Batch:0.794 | Total:0:01:37 | ETA:0:00:11 | Loss:0.253877439035857 | top1:89.08025360107422
131/135 Data:0.006 | Batch:0.780 | Total:0:01:44 | ETA:0:00:04 | Loss:0.2539118615953067 | top1:89.0848617553711
16/16 Data:0.002 | Batch:0.129 | Total:0:00:13 | ETA:0:00:00 | Loss:0.17204182805159154 | top1:92.97435760498047

Epoch: [182 | 200] LR: 0.000049
1/135 Data:3.009 | Batch:3.519 | Total:0:00:03 | ETA:0:07:52 | Loss:0.2792181074619293 | top1:87.890625
11/135 Data:0.006 | Batch:0.611 | Total:0:00:10 | ETA:0:02:06 | Loss:

51/135 Data:0.016 | Batch:0.782 | Total:0:00:44 | ETA:0:01:15 | Loss:0.24927653606031455 | top1:89.11611938476562
61/135 Data:0.011 | Batch:0.803 | Total:0:00:51 | ETA:0:00:58 | Loss:0.24929835097711595 | top1:89.16495513916016
71/135 Data:0.006 | Batch:0.776 | Total:0:00:59 | ETA:0:00:50 | Loss:0.2505658577445527 | top1:89.08175659179688
81/135 Data:0.007 | Batch:0.779 | Total:0:01:07 | ETA:0:00:42 | Loss:0.24928897784815895 | top1:89.11554718017578
91/135 Data:0.006 | Batch:0.707 | Total:0:01:15 | ETA:0:00:35 | Loss:0.24922577021541176 | top1:89.12259674072266
101/135 Data:0.002 | Batch:0.630 | Total:0:01:20 | ETA:0:00:21 | Loss:0.24990130311781816 | top1:89.08956909179688
111/135 Data:0.002 | Batch:0.632 | Total:0:01:28 | ETA:0:00:18 | Loss:0.2510554187738144 | top1:89.0801010131836
121/135 Data:0.613 | Batch:1.204 | Total:0:01:37 | ETA:0:00:13 | Loss:0.2509701966254179 | top1:89.09962463378906
131/135 Data:0.004 | Batch:0.616 | Total:0:01:44 | ETA:0:00:04 | Loss:0.2514508553133666 

21/135 Data:0.005 | Batch:0.525 | Total:0:00:24 | ETA:0:01:53 | Loss:0.2568525891928446 | top1:88.54167175292969
31/135 Data:0.004 | Batch:0.560 | Total:0:00:34 | ETA:0:01:41 | Loss:0.2549120072395571 | top1:88.83568572998047
41/135 Data:0.019 | Batch:0.544 | Total:0:00:42 | ETA:0:01:18 | Loss:0.2501077942731904 | top1:89.15776824951172
51/135 Data:0.003 | Batch:0.538 | Total:0:00:53 | ETA:0:01:34 | Loss:0.25136227116865273 | top1:89.05101776123047
61/135 Data:0.004 | Batch:0.521 | Total:0:01:01 | ETA:0:01:00 | Loss:0.24981064337198852 | top1:89.13933563232422
71/135 Data:0.004 | Batch:0.543 | Total:0:01:15 | ETA:0:01:31 | Loss:0.2490507341186765 | top1:89.16978454589844
81/135 Data:0.004 | Batch:0.525 | Total:0:01:23 | ETA:0:00:45 | Loss:0.25015829816276647 | top1:89.12036895751953
91/135 Data:0.006 | Batch:0.537 | Total:0:01:34 | ETA:0:00:47 | Loss:0.251440602016973 | top1:89.10327911376953
101/135 Data:0.004 | Batch:0.522 | Total:0:01:43 | ETA:0:00:33 | Loss:0.2514312687781778 | top

16/16 Data:0.001 | Batch:0.116 | Total:0:00:13 | ETA:0:00:00 | Loss:0.17251949680921358 | top1:92.9358901977539

Epoch: [196 | 200] LR: 0.000016
1/135 Data:4.222 | Batch:4.743 | Total:0:00:04 | ETA:0:10:36 | Loss:0.2164965569972992 | top1:91.2109375
11/135 Data:0.004 | Batch:0.523 | Total:0:00:12 | ETA:0:02:26 | Loss:0.23189714415506882 | top1:90.07457733154297
21/135 Data:0.938 | Batch:1.429 | Total:0:00:20 | ETA:0:01:34 | Loss:0.23847954188074386 | top1:89.88095092773438
31/135 Data:0.004 | Batch:0.525 | Total:0:00:27 | ETA:0:01:13 | Loss:0.24380667699921516 | top1:89.69883728027344
41/135 Data:0.700 | Batch:1.220 | Total:0:00:34 | ETA:0:01:11 | Loss:0.24569914326435183 | top1:89.657958984375
51/135 Data:0.003 | Batch:0.528 | Total:0:00:41 | ETA:0:00:57 | Loss:0.24863807331113255 | top1:89.52206420898438
61/135 Data:0.315 | Batch:0.848 | Total:0:00:48 | ETA:0:00:54 | Loss:0.24943667308228915 | top1:89.48513793945312
71/135 Data:0.002 | Batch:0.526 | Total:0:00:56 | ETA:0:00:47 | Loss

111/135 Data:0.103 | Batch:0.634 | Total:0:01:22 | ETA:0:00:18 | Loss:0.2504182526388684 | top1:89.29652404785156
121/135 Data:0.004 | Batch:0.558 | Total:0:01:29 | ETA:0:00:11 | Loss:0.2499251402606649 | top1:89.30784606933594
131/135 Data:0.003 | Batch:0.529 | Total:0:01:36 | ETA:0:00:03 | Loss:0.2503781525688317 | top1:89.27719116210938
16/16 Data:0.002 | Batch:0.119 | Total:0:00:18 | ETA:0:00:00 | Loss:0.1724534850472059 | top1:92.96153259277344
